In [1]:
#---Importing libraries---
import pandas as pd
import numpy as np
import time
import re
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [46]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
import xgboost as xgb
from sklearn.multioutput import MultiOutputClassifier

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler, OneHotEncoder,OrdinalEncoder

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# File location and type
df = pd.read_csv('/content/drive/MyDrive/Colab/supply_risk/clip_3.csv')
df.head(5)

,Vendor,Plant,Requested_Delv_Week,Actual_Delv_Week,SQOPE500,VGU_Age,VendorCountry,Avg_Delv_Date_diff,OTIF,QTY_Requested,...,Clip_Score,SCAR_Risk,Delivery_status,Material,BU,Max_Inv_qty,Min_Inv_qty,Safety_Stock,Daily_Moving_Stock,SS_Hit_Flag
0,100052951.0,US01,40.0,40.0,Top500,5.0,US,-1.0,1.0,1400.0,...,1.0,1.0,0.0,706083.0,Diagnostic X Ray Equipment-0960,0,0,0,1,0.0
1,100052951.0,US01,47.0,47.0,Top500,5.0,US,-1.0,1.0,1400.0,...,1.0,1.0,0.0,706100.0,Diagnostic X Ray Equipment-0960,0,0,0,1,0.0
2,100052951.0,US01,49.0,40.0,Top500,5.0,US,-45.0,0.0,700.0,...,0.0,1.0,1.0,706100.0,Diagnostic X Ray Equipment-0960,0,0,0,0,0.0
3,100052951.0,US01,53.0,53.0,Top500,5.0,US,0.0,1.0,1000.0,...,1.0,1.0,0.0,706100.0,Diagnostic X Ray Equipment-0960,0,0,0,1,0.0
4,100052951.0,US01,6.0,53.0,Top500,5.0,US,-23.0,0.0,700.0,...,0.0,1.0,1.0,706100.0,Diagnostic X Ray Equipment-0960,0,0,0,0,0.0


In [5]:
df.dtypes

Vendor                 float64
Plant                   object
Requested_Delv_Week    float64
Actual_Delv_Week       float64
SQOPE500                object
VGU_Age                float64
VendorCountry           object
Avg_Delv_Date_diff     float64
OTIF                   float64
QTY_Requested          float64
Qty_Overdelivered      float64
QTY_Received           float64
Clip_Score             float64
SCAR_Risk              float64
Delivery_status        float64
Material                object
BU                      object
Max_Inv_qty             object
Min_Inv_qty             object
Safety_Stock            object
Daily_Moving_Stock      object
SS_Hit_Flag            float64
dtype: object

In [6]:
df.columns

Index(['Vendor', 'Plant', 'Requested_Delv_Week', 'Actual_Delv_Week',
       'SQOPE500', 'VGU_Age', 'VendorCountry', 'Avg_Delv_Date_diff', 'OTIF',
       'QTY_Requested', 'Qty_Overdelivered', 'QTY_Received', 'Clip_Score',
       'SCAR_Risk', 'Delivery_status', 'Material', 'BU', 'Max_Inv_qty',
       'Min_Inv_qty', 'Safety_Stock', 'Daily_Moving_Stock', 'SS_Hit_Flag'],
      dtype='object')

In [7]:
df = df.dropna(how='any',axis=0)

In [38]:
pred_df = df[['Vendor', 'Plant', 'Material', 'Requested_Delv_Week','SQOPE500',
       'VGU_Age', 'VendorCountry','QTY_Requested','Clip_Score','SCAR_Risk', 'Delivery_status','BU','SS_Hit_Flag']]
cat=['Vendor', 'Plant','SQOPE500','VendorCountry','Clip_Score','SCAR_Risk', 'Material', 'BU']
for i in cat:
    pred_df[i] = pred_df[i].astype('category')
numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])
categorical_transformer = OneHotEncoder(handle_unknown='ignore')
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, cat)])
X = pred_df.drop(["Delivery_status"], axis=1)
X = pred_df.drop(["SS_Hit_Flag"], axis=1)
Y = pred_df[['Delivery_status','SS_Hit_Flag']]

In [39]:
# Test train split from the data with 20% reserved for testing
x_train, x_test, y_train, y_test = train_test_split(X,Y, train_size=0.8, random_state=12)

In [22]:
# Logistic Regression
lr = Pipeline(steps=[('preprocessor', preprocessor),
                     ('lr', MultiOutputClassifier(LogisticRegression()))])
lr.fit(x_train, y_train)
print("model Train score: %.3f" % lr.score(x_train, y_train))
print("model Test score: %.3f" % lr.score(x_test, y_test))

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Vendor', 'Plant',
                                                   'SQOPE500', 'VendorCountry',
                                                   'Clip_Score', 'SCAR_Risk',
                                                   'Material', 'BU'])])),
                ('lr', MultiOutputClassifier(estimator=LogisticRegression()))])

model Train score: 0.947
model Test score: 0.947


In [26]:
y_test.head(5)

,Delivery_status,SS_Hit_Flag
101691,0.0,0.0
370976,0.0,0.0
267159,0.0,0.0
331995,0.0,1.0
193851,1.0,0.0


In [40]:
pred_df2 = df[['Plant','Requested_Delv_Week','SQOPE500','VGU_Age', 'VendorCountry','QTY_Requested','Delivery_status','SS_Hit_Flag']]
cat=['Plant','SQOPE500','VendorCountry']
for i in cat:
    pred_df2[i] = pred_df2[i].astype('category')
numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])
categorical_transformer = OneHotEncoder(handle_unknown='ignore')
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, cat)])
X = pred_df2.drop(["Delivery_status"], axis=1)
X = pred_df2.drop(["SS_Hit_Flag"], axis=1)
Y = pred_df2[['Delivery_status','SS_Hit_Flag']]

In [41]:
# Test train split from the data with 20% reserved for testing
x_train, x_test, y_train, y_test = train_test_split(X,Y, train_size=0.8, random_state=12)

In [42]:
# Logistic Regression
lr = Pipeline(steps=[('preprocessor', preprocessor),
                     ('lr', MultiOutputClassifier(LogisticRegression()))])
lr.fit(x_train, y_train)
print("model Train score: %.3f" % lr.score(x_train, y_train))
print("model Test score: %.3f" % lr.score(x_test, y_test))

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Plant', 'SQOPE500',
                                                   'VendorCountry'])])),
                ('lr', MultiOutputClassifier(estimator=LogisticRegression()))])

model Train score: 0.896
model Test score: 0.898


In [43]:
# Random Forest Regression
rf = Pipeline(steps=[('preprocessor', preprocessor),
                     ('rf', MultiOutputClassifier(RandomForestClassifier(random_state=0)))])
rf.fit(x_train, y_train)
print("model Train score: %.3f" % rf.score(x_train, y_train))
print("model Test score: %.3f" % rf.score(x_test, y_test))

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Plant', 'SQOPE500',
                                                   'VendorCountry'])])),
                ('rf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(random_state=0)))])

model Train score: 0.898
model Test score: 0.900


In [45]:
# XGBoost Regression
xg = Pipeline(steps=[('preprocessor', preprocessor),
                     ('xg', MultiOutputClassifier(xgb.XGBClassifier()))])
xg.fit(x_train, y_train)
print("model Train score: %.3f" % xg.score(x_train, y_train))
print("model Test score: %.3f" % xg.score(x_test, y_test))

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Plant', 'SQOPE500',
                                                   'VendorCountry'])])),
                ('xg',
                 MultiOutputClassifier(estimator=XGBClassifier(base_score=None,
                                                               booster=None,
                                                               callbacks=None,
                                                               colsample_bylevel=None,
                                                               colsample_bynode=None,
                                                               colsample_bytree=None,
                                                               early_stopping_rounds=None,
                                                               enable_ca...
                                                               gpu_id=None,
                                                               grow_policy=None,
                                                               importance_type=None,
                                                               interaction_constraints=None,
                                                               learning_rate=None,
                                                               max_bin=None,
                                                               max_cat_threshold=None,
                                                               max_cat_to_onehot=None,
                                                               max_delta_step=None,
                                                               max_depth=None,
                                                               max_leaves=None,
                                                               min_child_weight=None,
                                                               missing=nan,
                                                               monotone_constraints=None,
                                                               n_estimators=100,
                                                               n_jobs=None,
                                                               num_parallel_tree=None,
                                                               predictor=None,
                                                               random_state=None, ...)))])

model Train score: 0.898
model Test score: 0.900
